# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Pipeline</span>

**Note**: This tutorial does not support Google Colab.

This is the first part of the quick start series of tutorials about Hopsworks Feature Store. As part of this first module, you will work with data related to credit card transactions. 
The objective of this tutorial is to demonstrate how to work with the **Hopworks Feature Store**  for online data with a goal of training and deploying a model that can predict fraudulent transactions.


## 🗒️ This notebook is divided in 3 sections:
1. Loading the data and feature engineeing,
2. Connect to the Hopsworks Feature Store,
3. Create feature groups and upload them to the Feature Store.

![tutorial-flow](../images/01_featuregroups.png)

First of all you will load the data and do some feature engineering on it.

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install -U 'hopsworks[python, great_expectations]' --quiet

In [ ]:
from math import radians

import numpy as np
import pandas as pd

from features import transactions_fraud

# Mute warnings
import warnings
warnings.filterwarnings("ignore")

First of all you will load the data and do some feature engineering on it.

## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

The data you will use comes from 2 different CSV files:

- `transactions.csv`: events containing information about when a credit card was used, such as a timestamp, location, and the amount spent. A boolean fraud_label variable (True/False) tells us whether a transaction was fraudulent or not.
- `profiles.csv`: credit card user information such as birthdate and city of residence.

In a production system, these CSV files would originate from separate data sources or tables, and probably separate data pipelines. **These files have a common credit card number column cc_num, which you will use later to join features together from the different datasets.**

Now, you can go ahead and load the data.

In [ ]:
# Read the profiles data from a CSV file
profiles_df = pd.read_csv(
    "https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_online/profiles.csv", 
    parse_dates=["birthdate"],
)

# Rename columns for clarity
profiles_df.columns = ["name", "gender", "mail", "birthdate", "City", "Country", "cc_num"]

# Select only the relevant columns ('cc_num' and 'gender')
profiles_df = profiles_df[["cc_num", "gender"]]

# Display the first three rows of the DataFrame
profiles_df.head(3)

In [ ]:
# Read the transactions data from a CSV file
trans_df = pd.read_csv(
    "https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_online/transactions.csv", 
    parse_dates=["datetime"],
)

# Display the first three rows of the DataFrame
trans_df.head(3)

In [ ]:
# Filter transactions DataFrame to include only rows with category "Cash Withdrawal"
trans_df = trans_df[trans_df.category == "Cash Withdrawal"].reset_index(level=0, drop=True)

# Fill missing values in the 'country' column with "US"
trans_df["country"] = trans_df["country"].fillna("US")

# Filter profiles DataFrame to include only rows with credit card numbers present in the filtered transactions DataFrame
profiles_df = profiles_df[profiles_df.cc_num.isin(trans_df.cc_num.unique())].reset_index(level=0, drop=True)

In [ ]:
# Sort the transactions DataFrame by 'datetime' and 'cc_num'
trans_df.sort_values(["datetime", "cc_num"], inplace=True)

---

## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

Fraudulent transactions can differ from regular ones in many different ways. Typical red flags would for instance be a large transaction volume/frequency in the span of a few hours. It could also be the case that elderly people in particular are targeted by fraudsters. To facilitate model learning you will create additional features based on these patterns. In particular, you will create two types of features:
1. **Features that aggregate data from different data sources**. This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.
2. **Features that aggregate data from multiple time steps**. An example of this could be the transaction frequency of a credit card in the span of a few hours, which is computed using a window function.

Let's start with the first category.

Now you are ready to start by computing the distance between consecutive transactions, lets call it `loc_delta`.
Here you will use the [Haversine distance](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html?highlight=haversine#sklearn.metrics.pairwise.haversine_distances) to quantify the distance between two longitude and latitude coordinates.

In [ ]:
# Use the prepare_transactions_fraud function to process the trans_df DataFrame
trans_df = transactions_fraud.prepare_transactions_fraud(trans_df)

# Display the first three rows of the modified DataFrame
trans_df.head(3)

## <span style="color:#ff5f27;">👮🏻‍♂️ Great Expectations </span>

In [ ]:
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

# Convert the 'trans_df' DataFrame to a Great Expectations DataFrame
ge_trans_df = ge.from_pandas(trans_df)

# Retrieve the expectation suite associated with the ge DataFrame
expectation_suite_transactions = ge_trans_df.get_expectation_suite()

# Set the expectation suite name to "transactions_suite"
expectation_suite_transactions.expectation_suite_name = "transactions_suite"

In [ ]:
# Check binary fraud_label column to be in set [0,1]
expectation_suite_transactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "fraud_label",
            "value_set": [0, 1],
        }
    )
)

# Check amount column to be not negative
expectation_suite_transactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "amount",
            "min_value": 0.0,
        }
    )
)

# Loop through specified columns ('tid', 'datetime', 'cc_num') and add expectations for null values
for column in ['tid', 'datetime', 'cc_num']:
    expectation_suite_transactions.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_null",
            kwargs={
                "column": column,
                "mostly": 0.0,
            }
        )
    )

In [ ]:
# Convert the 'profiles_df' DataFrame to a Great Expectations DataFrame
ge_profiles_df = ge.from_pandas(profiles_df)

# Retrieve the expectation suite associated with the ge DataFrame
expectation_suite_profiles = ge_profiles_df.get_expectation_suite()

# Set the expectation suite name to "profiles_suite"
expectation_suite_profiles.expectation_suite_name = "profiles_suite"

In [ ]:
# Check binary gender column to be in set ['M', 'F']
expectation_suite_profiles.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "gender",
            "value_set": ['M', 'F'],
        }
    )
)
# Check for Nulls
expectation_suite_profiles.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_null",
            kwargs={
                "column": 'cc_num',
                "mostly": 0.0,
            }
        )
    )

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

### <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

A [feature group](https://docs.hopsworks.ai/latest/concepts/fs/feature_group/fg_overview/) can be seen as a collection of conceptually related features. In this case, you will create a feature group for the transaction data and a feature group for the windowed aggregations on the transaction data. Both will have `cc_num` as primary key, which will allow you to join them when creating a dataset in the next tutorial.

Feature groups can also be used to define a namespace for features. For instance, in a real-life setting you would likely want to experiment with different window lengths. In that case, you can create feature groups with identical schema for each window length. 

Before you can create a feature group you need to connect to Hopsworks feature store.

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

To create a feature group you need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [ ]:
# Get or create the 'transactions_fraud_online_fg' feature group
trans_fg = fs.get_or_create_feature_group(
    name="transactions_fraud_online_fg",
    version=1,
    description="Transaction data",
    primary_key=['cc_num'],
    event_time='datetime',
    online_enabled=True,
    expectation_suite=expectation_suite_transactions,
)

Here you have also set `online_enabled=True`, which enables low latency access to the data. A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

At this point, you have only specified some metadata for the feature group. It does not store any data or even have a schema defined for the data. To make the feature group persistent you need to populate it with its associated data using the `insert` function.

In [ ]:
# Insert data into feature group
trans_fg.insert(trans_df)
print('✅ Done!')

In [ ]:
# Update feature descriptions
feature_descriptions = [
    {"name": "tid", "description": "Transaction id"},
    {"name": "datetime", "description": "Transaction time"},
    {"name": "cc_num", "description": "Number of the credit card performing the transaction"},
    {"name": "amount", "description": "Dollar amount of the transaction"},
    {"name": "country", "description": "Country in which the transaction was made"},
    {"name": "fraud_label", "description": "Whether the transaction was fraudulent or not"},
    {"name": "loc_delta_t_minus_1", "description": "Location of previous transaction"},
    {"name": "time_delta_t_minus_1", "description": "Time of previous transaction"},    
]

for desc in feature_descriptions: 
    trans_fg.update_feature_description(desc["name"], desc["description"])

You can move on and do the same thing for the profile and label feature groups.

In [ ]:
# Get or create the 'profile_fraud_online_fg' feature group
profile_fg = fs.get_or_create_feature_group(
    name="profile_fraud_online_fg",
    version=1,
    description="Credit card holder demographic data",
    primary_key=['cc_num'],
    online_enabled=True,
    expectation_suite=expectation_suite_profiles,
)
# Insert data into feature group
profile_fg.insert(profiles_df)
print('✅ Done!')

In [ ]:
# Update feature descriptions
feature_descriptions = [
    {"name": "cc_num", "description": "Number of the credit card performing the transaction"},
    {"name": "gender", "description": "Gender of the credit card holder"},
]

for desc in feature_descriptions: 
    profile_fg.update_feature_description(desc["name"], desc["description"])

Click on the hyperlink printed in the cell output above to inspect your feature group in the UI.

![fg-overview](../images/fg_overview.gif)

### <span style="color:#ff5f27;"> 👓  Exploration</span>
In the Hopsworks feature store, the metadata allows for multiple levels of explorations and review. Here you will explore a few of those capacities. 

### <span style="color:#ff5f27;"> 🔎 Search</span>
Using the search function in the UI, you can query any aspect of the feature groups, feature_view and training data that was previously created.

### <span style="color:#ff5f27;"> 📊 Statistics</span>
You can also enable statistics in one or all the feature groups.

In [ ]:
trans_fg = fs.get_or_create_feature_group("transactions_fraud_online_fg", version=1)
trans_fg.statistics_config = {
    "enabled": True,
    "histograms": True,
    "correlations": True,
}

trans_fg.update_statistics_config()
trans_fg.compute_statistics()

![fg-statistics](../images/fg_statistics.gif)


### ⛓️ <b> Lineage </b> 
In all the feature groups and feature view you can look at the relation between each abstractions; what feature group created which training dataset and that is used in which model.
This allows for a clear undestanding of the pipeline in relation to each element. 

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02 Training Pipeline </span>

In the following notebook you will use our feature groups to create a dataset you can train a model on.